In [2]:
# 导入视觉处理库 OpenCV
import cv2
# 导入数据处理库 numpy
import numpy as np

In [3]:
# 导入线程库threading
import threading
# 导入python的标准库time，提供与时间相关的函数
import time

In [4]:
# 导入用于在Jupyter中显示视频流的display库， IPython 的一个子模块，用于在 IPython 环境中显示各种类型的数据，包括图像、视频、音频、HTML 等
from IPython.display import display
# ipywidgets 是一个用于在 Jupyter Notebook 和 JupyterLab 中创建交互式控件（例如按钮、滑块、文本框、图像显示等）的库
import ipywidgets as widgets

In [5]:
# 相机内参矩阵
# matlab标定结果
# fx = 1495.79530974435
# fy = 1497.03409135696
# cx = 1531.82131315543
# cy = 940.869387256159

# OptiCalib标定结果(500w摄像头)
# 分辨率
# desired_width = 2592
# desired_height = 1944
# fx = 2106.3117386975
# fy = 2107.7631595681
# cx = 1540.6458072028
# cy = 917.3620404505
# k1 =  0.0809276902
# k2 = -0.2549836212
# p1 = -0.0008678516
# p2 = 0.0005847853
# k3 = 0.0

# OptiCalib标定结果(200w摄像头)
fx = 1462.933715
fy = 1472.343506
cx = 969.398521
cy = 545.531149 


intrinsic_matrix = np.array([[fx, 0, cx],
                             [0, fy, cy],
                             [0, 0, 1]], dtype=np.float32)

# 替换为你的实际畸变系数 (k1, k2, p1, p2, k3)(200w摄像头)
k1 =  0.057745
k2 = -0.117250
p1 = -0.001262
p2 = 0.001061
k3 = 0.0

distortion_coefficients = np.array([k1, k2, p1, p2, k3], dtype=np.float32)

In [9]:
# 打开摄像头
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("无法打开摄像头")
    exit()

In [10]:
# 设置分辨率
desired_width = 1920
desired_height = 1080
cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

True

In [19]:
# 释放摄像头
cap.release()

In [11]:
# 创建用于在 Jupyter 中显示的图像控件
image_widget_original = widgets.Image(format='jpeg')
image_widget_undistorted = widgets.Image(format='jpeg')
display(widgets.HBox([image_widget_original, image_widget_undistorted]))

In [18]:
# 控制视频流的标志
running = False

In [ ]:
# 获取制定大小的标定板中黑色棋盘格子的尺寸
def measure_and_display_top_left_square_size(image, pattern_size=(11, 8)):
    """
    Measures the pixel dimensions of the top-left black square in a
    checkerboard image and displays the measurements on the image.

    Args:
        image (numpy.ndarray): The input checkerboard image.
        pattern_size (tuple): The inner dimensions of the checkerboard
                              (width, height).

    Returns:
        numpy.ndarray: The input image with the top-left square dimensions
                       displayed on it. Returns the original image if the
                       checkerboard is not found.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

    if ret and corners is not None and len(corners) == pattern_size[0] * pattern_size[1]:
        corners = corners.reshape(-1, 2)

        # Assuming corners are ordered, the top-left black square's corners
        # will correspond to the first four detected inner corners.
        # The order might vary slightly depending on the detection algorithm,
        # but we'll try to infer based on the grid structure.

        # Find the minimum x and y coordinates to approximate the top-left corner
        min_x = np.min(corners[:, 0])
        min_y = np.min(corners[:, 1])

        # Try to find the four corners of the top-left square
        top_left_sq_corners = []
        # Look for corners close to the minimum x and y
        for corner in corners:
            if np.isclose(corner[0], min_x, atol=10) and np.isclose(corner[1], min_y, atol=10):
                top_left_sq_corners.append(corner)
            elif np.isclose(corner[0], min_x + (corners[1] - min_y)[0] if len(corners) > pattern_size[0] else min_x + 30, atol=20) and np.isclose(corner[1], min_y, atol=10):
                top_left_sq_corners.append(corner)
            elif np.isclose(corner[0], min_x, atol=10) and np.isclose(corner[1], min_y + (corners[pattern_size[0]] - min_y)[0] if len(corners) > pattern_size[0] else min_y + 30, atol=20):
                top_left_sq_corners.append(corner)
            elif np.isclose(corner[0], min_x + (corners[1] - min_y)[0] if len(corners) > pattern_size[0] else min_x + 30, atol=20) and np.isclose(corner[1], min_y + (corners[pattern_size[0]] - min_y)[0] if len(corners) > pattern_size[0] else min_y + 30, atol=20):
                top_left_sq_corners.append(corner)
            if len(top_left_sq_corners) == 4:
                break

        if len(top_left_sq_corners) == 4:
            # Sort the corners to get a consistent order (e.g., top-left, top-right, bottom-right, bottom-left)
            top_left_sq_corners = np.array(sorted(top_left_sq_corners, key=lambda x: (x[1], x[0])))

            tl = top_left_sq_corners[0]
            tr = min(top_left_sq_corners, key=lambda x: (np.linalg.norm(x - tl - [100, 0]))) # Approximate right
            bl = min(top_left_sq_corners, key=lambda x: (np.linalg.norm(x - tl - [0, 100]))) # Approximate bottom

            width = np.linalg.norm(tr - tl)
            height = np.linalg.norm(bl - tl)

            text_width = f"Width: {width:.2f} px"
            text_height = f"Height: {height:.2f} px"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            font_thickness = 2
            text_color = (0, 255, 0)  # Green
            text_tl_width = (int(tl[0]), int(tl[1] - 20))
            text_tl_height = (int(tl[0]), int(tl[1] - 45))

            cv2.putText(image, text_width, text_tl_width, font, font_scale, text_color, font_thickness, cv2.LINE_AA)
            cv2.putText(image, text_height, text_tl_height, font, font_scale, text_color, font_thickness, cv2.LINE_AA)

            # Draw a rectangle around the approximate top-left square (for visualization)
            if tr is not None and bl is not None:
                br = (int(bl[0] + (tr[0] - tl[0])), int(tr[1] + (bl[1] - tl[1])))
                cv2.rectangle(image, (int(tl[0]), int(tl[1])), br, (0, 0, 255), 2) # Red rectangle

    return image

In [ ]:
def detect_and_display_square_corners(image):
  """
  检测包含多个黑色正方形的图像中的角点，并在图像上显示角点。
 
  Args:
  image (numpy.ndarray): 输入图像。
 
  Returns:
  numpy.ndarray: 标记了检测到的角点的输入图像。
  """
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
  # 使用 goodFeaturesToTrack 进行角点检测
  corners = cv2.goodFeaturesToTrack(gray, maxCorners=1000, qualityLevel=0.01, minDistance=10)
 
  if corners is not None:
      corners = np.int8(corners)  # 转换为整数以便绘制
  for i in corners:
      x, y = i.ravel()
      cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # 用红色标记角点
 
  return image

In [16]:
def capture_and_display():
    while running:
        ret, frame = cap.read()
        if not ret:
            print("无法获取帧")
            break

        # 矫正图像
        undistorted_frame = cv2.undistort(frame, intrinsic_matrix, distortion_coefficients)
        # measure_cube_size_auto_image(undistorted_frame)
        # 将原始帧转换为 JPEG 并更新图像控件
        _, frame_jpeg = cv2.imencode('.jpeg', frame)
        image_widget_original.value = frame_jpeg.tobytes()

        # 将矫正后的帧转换为 JPEG 并更新图像控件
        _, undistorted_frame_jpeg = cv2.imencode('.jpeg', undistorted_frame)
        image_widget_undistorted.value = undistorted_frame_jpeg.tobytes()

        time.sleep(0.03)  # 控制帧率 (可选)
    print("Thread Stop")

In [17]:
# 在一个新线程中运行视频捕获和显示
thread = threading.Thread(target=capture_and_display)
thread.start()

Thread Stop


In [ ]:
# 结束线程
thread.join()

In [ ]:
# 使用标定后的相机参数估计图像中棋盘格子的像素尺寸。
def estimate_checkerboard_square_size_pixels_calibrated(image, pattern_size=(10, 7), real_square_size_mm=24.0, cameraMatrix=None, distCoeffs=None):
    """
    使用标定后的相机参数估计图像中棋盘格子的像素尺寸。

    Args:
        image (numpy.ndarray): 输入的棋盘格图像。
        pattern_size (tuple): 棋盘格的内部角点数量 (宽度, 高度)。
                             对于 11x8 的格子，内部角点是 (10, 7)。
        real_square_size_mm (float): 棋盘格子的实际尺寸，单位毫米。
        cameraMatrix (numpy.ndarray): 标定后的相机内参矩阵 (3x3)。
        distCoeffs (numpy.ndarray): 标定后的畸变系数。

    Returns:
        tuple: 包含平均像素宽度和平均像素高度的元组 (avg_pixel_width, avg_pixel_height)，
               如果未检测到棋盘格或未提供标定参数，则返回 None。
    """
    if cameraMatrix is None or distCoeffs is None:
        print("警告：未提供相机标定参数。将尝试直接从图像估计像素尺寸（可能不准确）。")
        return estimate_checkerboard_square_size_pixels(image, pattern_size)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

    if ret and corners is not None:
        corners = corners.reshape(-1, 2)

        all_pixel_widths = []
        all_pixel_heights = []

        # 计算相邻角点之间的像素距离
        for i in range(pattern_size[1]):
            for j in range(pattern_size[0] - 1):
                p1 = corners[i * pattern_size[0] + j]
                p2 = corners[i * pattern_size[0] + j + 1]
                pixel_width = np.linalg.norm(p1 - p2)
                all_pixel_widths.append(pixel_width)

        for i in range(pattern_size[1] - 1):
            for j in range(pattern_size[0]):
                p1 = corners[i * pattern_size[0] + j]
                p2 = corners[(i + 1) * pattern_size[0] + j]
                pixel_height = np.linalg.norm(p1 - p2)
                all_pixel_heights.append(pixel_height)

        if all_pixel_widths and all_pixel_heights:
            avg_pixel_width = np.mean(all_pixel_widths)
            avg_pixel_height = np.mean(all_pixel_heights)

            # 像素尺寸 (mm/像素)
            pixel_size_width_mm_per_pixel = real_square_size_mm / avg_pixel_width
            pixel_size_height_mm_per_pixel = real_square_size_mm / avg_pixel_height

            print(f"估计的平均像素宽度: {avg_pixel_width:.2f} 像素")
            print(f"估计的平均像素高度: {avg_pixel_height:.2f} 像素")
            print(f"估计的水平方向像素尺寸: {pixel_size_width_mm_per_pixel:.4f} mm/像素")
            print(f"估计的垂直方向像素尺寸: {pixel_size_height_mm_per_pixel:.4f} mm/像素")

            return avg_pixel_width, avg_pixel_height

    else:
        print("未检测到棋盘格角点。")
        return None



if __name__ == '__main__':
    # 替换为你的棋盘格图像路径
    image_path = 'your_checkerboard_image.jpg'
    img = cv2.imread(image_path)

    if img is not None:
        # 你的棋盘格尺寸
        pattern_size = (10, 7)  # 11x8 的格子有 10x7 个内部角点
        real_square_size_mm = 24.0

        # 如果你已经标定过相机，请加载你的相机矩阵和畸变系数
        # camera_matrix = np.load('camera_matrix.npy')
        # dist_coeffs = np.load('distortion_coefficients.npy')
        camera_matrix = None  # 如果你没有标定参数，设置为 None
        dist_coeffs = None    # 如果你没有标定参数，设置为 None

        average_pixel_size = estimate_checkerboard_square_size_pixels_calibrated(
            img, pattern_size, real_square_size_mm, camera_matrix, dist_coeffs
        )

        if average_pixel_size:
            avg_width_px, avg_height_px = average_pixel_size
            # 你现在可以根据实际尺寸计算每像素代表的毫米数
            pixel_width_mm = real_square_size_mm / avg_width_px
            pixel_height_mm = real_square_size_mm / avg_height_px
            print(f"估计的水平方向像素尺寸: {pixel_width_mm:.4f} mm/像素")
            print(f"估计的垂直方向像素尺寸: {pixel_height_mm:.4f} mm/像素")

    else:
        print(f"无法加载图像: {image_path}")

In [9]:
def estimate_checkerboard_square_size_pixels(image, pattern_size=(10, 7)):
    """
    直接从图像估计棋盘格子的像素尺寸（未进行相机标定）。
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

    if ret and corners is not None:
        corners = corners.reshape(-1, 2)
        all_pixel_widths = []
        all_pixel_heights = []

        for i in range(pattern_size[1]):
            for j in range(pattern_size[0] - 1):
                p1 = corners[i * pattern_size[0] + j]
                p2 = corners[i * pattern_size[0] + j + 1]
                all_pixel_widths.append(np.linalg.norm(p1 - p2))

        for i in range(pattern_size[1] - 1):
            for j in range(pattern_size[0]):
                p1 = corners[i * pattern_size[0] + j]
                p2 = corners[(i + 1) * pattern_size[0] + j]
                all_pixel_heights.append(np.linalg.norm(p1 - p2))

        if all_pixel_widths and all_pixel_heights:
            avg_width = np.mean(all_pixel_widths)
            avg_height = np.mean(all_pixel_heights)
            print(f"直接估计的平均像素宽度: {avg_width:.2f} 像素")
            print(f"直接估计的平均像素高度: {avg_height:.2f} 像素")
            return avg_width, avg_height
    return None

In [32]:
capture_and_display()

正方体边界框宽度 (像素): 1531 像素
正方体边界框高度 (像素): 2044 像素
估计的正方体宽度: 394.71 毫米
估计的正方体高度: 529.42 毫米
正方体边界框宽度 (像素): 1927 像素
正方体边界框高度 (像素): 1530 像素
估计的正方体宽度: 496.81 毫米
估计的正方体高度: 396.29 毫米
正方体边界框宽度 (像素): 1531 像素
正方体边界框高度 (像素): 2044 像素
估计的正方体宽度: 394.71 毫米
估计的正方体高度: 529.42 毫米
正方体边界框宽度 (像素): 1532 像素
正方体边界框高度 (像素): 2043 像素
估计的正方体宽度: 394.97 毫米
估计的正方体高度: 529.16 毫米
正方体边界框宽度 (像素): 1732 像素
正方体边界框高度 (像素): 1530 像素
估计的正方体宽度: 446.54 毫米
估计的正方体高度: 396.29 毫米
正方体边界框宽度 (像素): 1531 像素
正方体边界框高度 (像素): 1924 像素
估计的正方体宽度: 394.71 毫米
估计的正方体高度: 498.34 毫米
正方体边界框宽度 (像素): 1531 像素
正方体边界框高度 (像素): 2044 像素
估计的正方体宽度: 394.71 毫米
估计的正方体高度: 529.42 毫米
正方体边界框宽度 (像素): 1927 像素
正方体边界框高度 (像素): 1529 像素
估计的正方体宽度: 496.81 毫米
估计的正方体高度: 396.03 毫米
正方体边界框宽度 (像素): 1531 像素
正方体边界框高度 (像素): 2044 像素
估计的正方体宽度: 394.71 毫米
估计的正方体高度: 529.42 毫米
正方体边界框宽度 (像素): 1930 像素
正方体边界框高度 (像素): 1530 像素
估计的正方体宽度: 497.58 毫米
估计的正方体高度: 396.29 毫米
正方体边界框宽度 (像素): 1927 像素
正方体边界框高度 (像素): 1530 像素
估计的正方体宽度: 496.81 毫米
估计的正方体高度: 396.29 毫米
正方体边界框宽度 (像素): 1531 像素
正方体边界框高度 (像素): 2044 像素
估计的正方体宽度

KeyboardInterrupt: 

In [31]:
# 假设你已经运行了棋盘格检测代码并得到了以下平均像素尺寸
avg_pixel_width_from_checkerboard = 93.09  # 替换为你的实际值
avg_pixel_height_from_checkerboard = 92.66 # 替换为你的实际值
real_square_size_mm = 24.0  # 棋盘格的实际尺寸

# 计算得到的毫米/像素值
mm_per_pixel_width = real_square_size_mm / avg_pixel_width_from_checkerboard
mm_per_pixel_height = real_square_size_mm / avg_pixel_height_from_checkerboard

def measure_cube_size_auto_image(image):
    """
    使用焦点检测自动获取白色正方体的边界框并测量其大小 (直接传入图像)。

    Args:
        image (numpy.ndarray): 白色正方体的图像 (NumPy 数组)。

    Returns:
        tuple: 包含估计的宽度和高度（以毫米为单位），如果未检测到有效边界框则返回 None。
    """
    if image is None or image.size == 0:
        print("传入的图像为空。")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 使用 goodFeaturesToTrack 进行角点检测
    corners = cv2.goodFeaturesToTrack(gray, maxCorners=500, qualityLevel=0.01, minDistance=10)

    if corners is not None and len(corners) >= 4:
        corners = np.int32(corners)
        # 获取所有角点的最小外接矩形
        rect = cv2.minAreaRect(corners)
        box = cv2.boxPoints(rect)
        box = np.int32(box)
        

        # 计算边界框的宽度和高度（可能会倾斜）
        width_pixels = int(np.linalg.norm(box[0] - box[1]))
        height_pixels = int(np.linalg.norm(box[1] - box[2]))

        estimated_width_mm = width_pixels * mm_per_pixel_width
        estimated_height_mm = height_pixels * mm_per_pixel_height

        print(f"正方体边界框宽度 (像素): {width_pixels} 像素")
        print(f"正方体边界框高度 (像素): {height_pixels} 像素")
        print(f"估计的正方体宽度: {estimated_width_mm:.2f} 毫米")
        print(f"估计的正方体高度: {estimated_height_mm:.2f} 毫米")

        # 绘制边界框和尺寸
        cv2.drawContours(image, [box], 0, (0, 255, 0), 2)
        text_width = f"Width: {estimated_width_mm:.2f} mm"
        text_height = f"Height: {estimated_height_mm:.2f} mm"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        text_color = (0, 255, 0)
        text_pos_width = (box[0][0], box[0][1] - 10 if box[0][1] - 10 > 10 else box[2][1] + 20)
        text_pos_height = (box[0][0], box[0][1] - 30 if box[0][1] - 30 > 10 else box[2][1] + 40)

        cv2.putText(image, text_width, text_pos_width, font, font_scale, text_color, font_thickness, cv2.LINE_AA)
        cv2.putText(image, text_height, text_pos_height, font, font_scale, text_color, font_thickness, cv2.LINE_AA)

        cv2.imshow("自动检测和测量的正方体", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return estimated_width_mm, estimated_height_mm
    else:
        print("未能检测到足够的角点来构建边界框。")
        return None